In [87]:
import glob
import time
import nltk
import collections
import sys
from math import *
import json

In [88]:
#If there are n terms in the query, we must iterate n times.
#For every term in the query, find the document length, average document length, and the 
    #frequency of the term in the document. Using the formula, add them all as the ranking
    #score and use it as a statistic for the document.
    
def calculateRSVd(queryList, finalResult):
    match = []
    RSVd = 0;
    for docID in finalResult:
        for word in queryList: 
            if word == 'and':
                pass
            else:
                match = calculateQueryWordRef(word, docID)
                docLength = match[0]
                termFrequencyInDoc = match[1]
                docFrequency = match[2]
                del match[:]
                step1 = (1 - b) + b * (docLength / avgDocLength)
                step2 = (k + 1) * termFrequencyInDoc / (k * step1 + termFrequencyInDoc)
                RSVdq = log(float(docCounter / docFrequency)) * step2
                
                RSVd = RSVd + RSVdq
                docRanking[docID] = RSVd
                
    return docRanking

In [89]:
def calculateQueryWordRef(key, docID):
    #Initalization
    match = []
    dictAid = {}
    fileName = "final_dict.json"
    termFrequencyInDoc = 0
    docFrequency = 0
    #Convert key to lowercase
    key = key.lower()
    with open(fileName, 'r') as json_file: 
        finalDict = json.load(json_file)
        dictAid = finalDict.get(key)
        if finalDict.has_key(key):
            termFrequencyInDoc = dictAid.get(docID)
            docFrequency = len(dictAid)
            
        match.append(docLength)
        match.append(termFrequencyInDoc)
        match.append(docFrequency)
    json_file.close()
    
    return match

In [90]:
#Open JSON file and do single query
def findMatch(key): 
    match = []
    fileName = "final_dict.json"
    #Convert key to lowercase
    key = key.lower()
    with open(fileName, 'r') as json_file: 
        json_dict = json.load(json_file)
        if json_dict.has_key(key):
            for docID in json_dict[key].keys():
                match.append(docID)
        else:
            print "There is no key named", key
    json_file.close()
    match.sort()
    return match

In [91]:
def sortMatch(list):
    newList = []
    for word in list: 
        word = int(word)
        newList.append(word)
    newList.sort()
    for word in newList: 
        word = "u'    " + str(word) + "   '"
    print 'list is', newList
    return newList

In [92]:
def singleWordQuery(queryList):
    newMatch = {}
    result_and = []
    for key in queryList:
        match = findMatch(key)
        newMatch[key] = sorted(match[key])
        print key, 'exists in : ', newMatch[key], 'docs'

In [93]:
def stopWords30(word):
    words30= ['',' ','a', 'about', 'above', 'across', 'after', 
              'again', 'against','all',
              'almost','alone','along','already','also',
              'although','always','among','an','and',
              'another','any','anybody','anyone','anything','anywhere','are',' ','areas',
              'around','as','\n']
    if word in words30: 
        x = True
        return x;
    else: 
        x = False
        return x
    
def stopWords150(word):
    words150 = ['',' ','\n','a', 'about', 'above', 'after', 'again', 'against', 'all', 
                'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'an', 
                'and', 'another', 'any', 'anybody', 'anyone', 'anything', 'anywhere', 'around', 
                'as', 'at', 'away', 'been', 'but', 'even', 'evenly', 'ever', 'every', 'everybody', 
                'everyone', 'everything', 'everywhere', 'for', 'had', 'has', 'have', 'he', 'her', 
                'here', 'herself', 'higher', 'highest', 'him', 'himself', 'his', 'how', 'however', 
                'if', 'in', 'interests', 'interesting', 'into', 'is', 'it', 'its', 'itself', 'just', 
                'last', 'me', 'might', 'more', 'most', 'mostly', 'mr', 'mrs', 'much', 'must', 'my', 'myself', 
                'of', 'off', 'often', 'on', 'once', 'only', 'or', 'other', 'others', 'our', 'out', 'over', 
                'second', 'seconds', 'seem', 'seemed', 'seeming', 'seems', 'sees', 'several', 'shall', 
                'she', 'should', 'since', 'so', 'some', 'somebody', 'someone', 'something', 'somewhere', 
                'still', 'such', 'than', 'that', 'the', 'their', 'them', 'then', 'there', 'therefore', 
                'these', 'they', 'things', 'this', 'those', 'though', 'thoughts', 'through', 'thus', 
                'to', 'too', 'until', 'us', 'very', 'was', 'ways', 'we', 'were', 'what', 'when', 'where', 
                'whether', 'which', 'while', 'who', 'whole', 'whose', 'why', 'will', 'with', 'within', 'without', 
                'would', 'yet', 'you', 'your', 'yours']
    if word in words150:
        x = True
        return x;
    else:
        x = False
        return x

In [94]:
def mergeTokenDict(dict1, dict2):
    for key in dict2.keys():
        if key in dict1.keys():
            for docID in dict2[key].keys():
                if docID in dict1[key].keys():
                    dict1[key][docID] += dict2[key][docID]
                else:
                    dict1[key][docID] = dict2[key][docID]
        else:
            dict1[key] = dict2[key]
    return dict1

            
def ranking(queryList, finalResult):
    if(len(finalResult) > 0):
        docRanking = calculateRSVd(queryList, finalResult)
        if len(docRanking) == 1:
            print docRanking
        else: 
            print sorted(docRanking.items(), lambda x, y: cmp(x[1], y[1]), reverse = True)
        docRanking.clear()

In [95]:
print '----------------------------Start the program-------------------------------'

tokendict = {}
docLenStore = {} 
docRanking = {} 
fileCounter =0
path = '/Users/shilpakancharla/Documents/reuters/data-files/'
delimite = ' '
sgmfiles = glob.glob(path + 'reut2-*.sgm')
memorylimit = 1024 *3
k = 1.2
b = 0.75
start = time.clock()
docCounter = 0  # count the number of doc
doclenCounterList = []; #store every doclen, in order to calculate the avgdoclen in convience

----------------------------Start the program-------------------------------


In [99]:
print '---Open the files---'
print '---Split the files---'
fileName = 'file' + str(fileCounter) + '.json' #change to json file to store the dictionary
target = open(fileName,'w')
target.truncate()
stopWords = 0
numbers = 0
word_flag = True;

for i in range(len(sgmfiles)):
    with open(sgmfiles[i],'r') as f:
        totalDocLen = 0;
        flag = False
        docLength = 0
        for line in f:
            line = line.replace('<', ' <');
            line = line.replace('>', '> ');
            line = line.replace('\n', ' ');
            line = line.replace('\t', ' ');
            line = line.replace('.','');
            line = line.replace(',',' ')
            line = line.replace('.',' ')
            line = line.replace('}',' ')
            line = line.replace('{',' ')
            line = line.replace('&',' ')
            line = line.replace('-',' ')
            line = line.replace('"',' ')
            line = line.replace(';',' ')
            line = line.replace("(",' ')
            line = line.replace(")",' ')
            line = line.replace('*','')
            line = line.replace(':',' ')
            line = line.replace('+',' ')
            line = line.replace('$',' ')
            line = line.replace("'",'')
            line = line.replace("=",' ')
            if('<REUTERS' in line):
                docIDPreviousOne = 0
                
                docID = line[line.find('NEWID'): ]
                docID = docID.replace('"','')
                docID = docID.replace('>','')
                docID = docID.replace('NEWID','')
                docID = docID.replace('=','')
                
                if(docID != docIDPreviousOne):
                    docCounter += 1 #calculate the total number of docs = (N)
                    #store the totalDocLen in the docLenStore
                docLenStore[docID] = totalDocLen
                totalDocLen = 0
                    #reset totalDocLen
            for word in line.split(' '):
                word = word.strip()
                if(word.lower() == '<body>' or word.lower() == '<title>'):
                    flag = True
                    pass
                elif(word.lower() == '</body>' or word.lower() == '</title>'):
                    flag = False
                    pass
                elif flag:
                    if(word==' ' or word == '' or '&#' in word or '/' in word or '[' in word or ']' in word or '@' in word or '=' in word or '?' in word or '#' in word or '&' in word ):
                        pass
                    elif(word.isdigit()): #remove numbers
                        numbers += 1
                        pass
                    elif (stopWords150(word)): #remove 150 stopwords
                        stopWords += 1
                        pass
                    else:
                        word = word.replace('<','')
                        word = word.replace('>','')
                        word = word.lower()
                        totalDocLen += 1
                        if word in tokendict:
                            if tokendict[word].has_key(docID):
                                tokendict[word][docID] += 1
                            else:
                                tokendict[word][docID] = 1
                        else:
                            tokendict[word] = {}
                            tokendict[word][docID] = 1
                            
                #Check memory size
                if ((sys.getsizeof(tokendict)/1024) >= memorylimit):
                    fileCounter = fileCounter + 1
                    tokendict = collections.OrderedDict(sorted(tokendict.items()))
                    fileName = "file" + str(fileCounter) +".json"
                    data_string = json.dumps(tokendict, encoding = 'latin-1')
                    target = open(fileName,'w')
                    target.write(data_string)
                    target.close()
                    tokendict.clear()
                    filename_doclenStore = "doclenStore.json"
                    data_string1 = json.dumps(docLenStore)
                    target1 = open(filename_doclenStore,'w')
                    target1.write(data_string1)
                    target1.close()

---Open the files---
---Split the files---


In [102]:
#if the block is not fulled in final,we need to get this part too
tokendict = collections.OrderedDict(sorted(tokendict.items()))

#Calculate average document length
totalLen = 0
for docID, docLength in docLenStore.items():
    totalLen += docLength
    avgdoclen = totalLen/len(docLenStore)
    print avgdoclen

#data_string = unicode(json.dumps(tokendict),errors = 'ignore')
data_string = json.dumps(tokendict,encoding = 'latin-1')
target = open(fileName,'w')
target.write(data_string)
target.close()
filename_doclenStore = "doclenStore.json"
data_string1 = json.dumps(docLenStore)
target1 = open(filename_doclenStore,'w')
target1.write(data_string1)
target1.close()
#---------------------merge blocks----------------------------------------------------------------------
#pay attention that when merge them ,need to check the term part
mergeCounter = 0
final_dict={}
#print '-------------file counter: ',fileCounter~~~
#merge each small dictionary(merge those json file)
for mergeCounter in range(fileCounter + 1):
    fileName = "file"+str(mergeCounter)+".json"
    with open(fileName,'r') as dict2:
        dict2 = json.load(dict2)
        final_dict = mergeTokenDict(final_dict,dict2)
        
#merge this dictionary docLenStore-->ignore
fileName = "final_dict.json"
data_string = json.dumps(final_dict)
target = open(fileName, 'w')
target.write(data_string)
target.close()

print '# of numbers deleted = ', numbers
print '# of stopWords deleted = ', stopWords

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


19
19
19
19
19
19
19
19
19
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
2

33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
3

44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
4

59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
6

72
72
72
72
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
74
7

84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85


ValueError: No JSON object could be decoded

In [103]:
#---------------------Query part------------------------------
match1 = {}
query_results = {}

finalResult = []# an empty smallList, in order to do the insection operation,and store the final_result in final
andFlag = False
orFlag = False
queryFlag = True
queryList = [] # store the query words

while queryFlag:
    query = raw_input("--------Please enter your Query-----------\n\n")
    singleResultList = []
    queryList = query.split(' ')
    print queryList
    
    #Single word query
    if len(queryList) == 1:
        finalResult = findMatch(query)
        print 'The result of', query,'is', finalResult
        ranking(queryList, finalResult)

    else:
        for word in query_list:
            if (word == 'and' or word == 'or'):
                pass
            else: #do single query for each word
                list_of_matchedItems = findMatch(word)
                singleResultList.append(list_of_matchedItems)
                print('word is:', word, findMatch(word))
                
        print 'Consumed time =', time.clock()-start,'ms'
        p = 0
        finalResult = singleResultList[0]
        for index in xrange(1, len(singleResultList)):
        
            finalResult = list(set(finalResult) & set(singleResultList[index]))
        print 'The final result is: ', finalResult
        ranking(queryList, finalResult)

--------Please enter your Query-----------

george bush
['george', 'bush']
Consumed time = 277.424494 ms


IndexError: list index out of range